# Cadastro e Reconhecimento Facial com `face_recognition`

Este notebook demonstra como **cadastrar** rostos (gerar e salvar **embeddings**) e depois **reconhecê‑los** em novas imagens usando a biblioteca [`face_recognition`](https://github.com/ageitgey/face_recognition). Os passos estão organizados em células separadas para facilitar o entendimento e a execução sequencial.

## 1. Instalação de dependências
Execute a célula abaixo **apenas** se você ainda não tem as bibliotecas instaladas no seu ambiente. No Google Colab, é necessário reiniciar o runtime após a instalação do `dlib`.


In [1]:
!pip install face_recognition opencv-python pillow tqdm

/bin/bash: /home/koiti/anaconda3/lib/libtinfo.so.6: no version information available (required by /bin/bash)


## 2. Importação de bibliotecas

In [2]:
import face_recognition
import cv2
import pickle
from pathlib import Path
from tqdm import tqdm


## 3. Cadastro de pessoas
Nesta etapa vamos:
1. Definir uma pasta com **imagens de referência** (uma ou mais por pessoa), chamada dataset_faces.
2. Para cada imagem, detectar o(s) rosto(s) e extrair o **encoding** de 128 dimensões.
3. Armazenar os encodings em um dicionário `{nome: [lista de encodings]}`.
4. Salvar o cadastro em disco (`encodings.pickle`).

O nome que aparecerá na hora do reconhecimento vem da chave do dicionário de encodings — 
e essa chave é criada a partir do nome da pasta onde estão as fotos de referência.

In [ ]:
# Caminho para a pasta de imagens de referência
dataset_dir = Path('dataset_faces')  # coloque suas imagens nesta pasta
assert dataset_dir.exists(), f'Pasta {dataset_dir} não encontrada.'

# Inicializa dicionário de encodings
encodings_dict = {}

# Percorre todas as imagens (suporta subpastas com o nome da pessoa)
for img_path in tqdm(list(dataset_dir.rglob('*.[jp][pn]g')), desc='Processando imagens'):
    name = img_path.parent.name  # supõe que a pasta é o nome da pessoa
    image = face_recognition.load_image_file(img_path)
    face_locations = face_recognition.face_locations(image, model='hog')  # ou 'cnn' se tiver GPU
    if len(face_locations) != 1:
        print(f'Aviso: {img_path} tem {len(face_locations)} rostos; ignorando.' if len(face_locations)!=1 else '')
        continue
    encoding = face_recognition.face_encodings(image, known_face_locations=face_locations)[0]
    encodings_dict.setdefault(name, []).append(encoding)

print(f'Pessoas cadastradas: {list(encodings_dict.keys())}')

# Salva em disco
with open('encodings.pickle', 'wb') as f:
    pickle.dump(encodings_dict, f)
print('Arquivo encodings.pickle salvo com sucesso!')


Ou, se preferir não criar pastas com os nomes das pessoas, e cadastras por apenas uma imagem mesmo:

In [3]:
from pathlib import Path
import pickle, face_recognition

# 1. Carrega a imagem cujo rosto será cadastrado
image = face_recognition.load_image_file('/home/koiti/Downloads/koititeste1.jpeg')
encoding = face_recognition.face_encodings(image)[0]

# 2. Pede o nome que servirá de rótulo
nome = input("Digite o nome da pessoa: ").strip()

# 3. Abre (ou cria) o arquivo de encodings
enc_file = Path('encodings.pickle')
if enc_file.exists():
    with enc_file.open('rb') as f:
        enc = pickle.load(f)
else:
    enc = {}                       # começa do zero

# 4. Atualiza o dicionário com o novo rosto
enc.setdefault(nome, []).append(encoding)

# 5. Salva de volta em disco
with enc_file.open('wb') as f:
    pickle.dump(enc, f)

print(f"Cadastro de {nome} atualizado.")


Cadastro de Koiti atualizado.


## 4. Reconhecimento em uma imagem
Nesta célula usamos o arquivo `encodings.pickle` para identificar rostos em uma imagem de teste.

In [6]:
from pathlib import Path
import pickle, cv2, face_recognition
# ───────────────────────────────────────────────────────────────────────
# Carrega encodings
with open('encodings.pickle', 'rb') as f:
    encodings_dict = pickle.load(f)

# Constrói listas para busca rápida
known_names, known_encodings = [], []
for name, encs in encodings_dict.items():
    known_names.extend([name] * len(encs))
    known_encodings.extend(encs)

# Imagem a ser reconhecida
test_img_path = Path('/home/koiti/Downloads/koititeste1.jpeg')
assert test_img_path.exists(), 'Imagem de teste não encontrada.'
image = face_recognition.load_image_file(test_img_path)

# Detecta e reconhece
face_locations = face_recognition.face_locations(image)
face_encodings = face_recognition.face_encodings(image, face_locations)

# Converte para BGR para exibir/salvar com OpenCV
image_bgr = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

for (top, right, bottom, left), face_encoding in zip(face_locations, face_encodings):
    matches = face_recognition.compare_faces(known_encodings, face_encoding, tolerance=0.45)
    name = "Desconhecido"
    face_distances = face_recognition.face_distance(known_encodings, face_encoding)
    if matches:
        best_match_index = face_distances.argmin()
        if matches[best_match_index]:
            name = known_names[best_match_index]

    # Desenha caixa e nome
    cv2.rectangle(image_bgr, (left, top), (right, bottom), (0, 255, 0), 2)
    cv2.rectangle(image_bgr, (left, bottom - 20), (right, bottom), (0, 255, 0), cv2.FILLED)
    cv2.putText(image_bgr, name, (left + 2, bottom - 5),
                cv2.FONT_HERSHEY_DUPLEX, 0.5, (0, 0, 0), 1)

# ─── Salvar sem sobrescrever ───────────────────────────────────────────
out_path = Path("recognized.jpg")
if out_path.exists():
    i = 1
    while True:
        candidate = out_path.with_stem(f"{out_path.stem}_{i}")  # recognized_1.jpg ...
        if not candidate.exists():
            out_path = candidate
            break
        i += 1

cv2.imwrite(str(out_path), image_bgr)
print(f"Resultado salvo em {out_path.name}")



Resultado salvo em recognized_1.jpg


## 5. Reconhecimento em tempo real via webcam (opcional)
Pressione **`q`** para encerrar a captura.

In [7]:
# ⚠️ Execute esta célula somente em ambiente local com webcam
video_capture = cv2.VideoCapture(0)

with open('encodings.pickle', 'rb') as f:
    encodings_dict = pickle.load(f)

known_names = []
known_encodings = []
for name, encs in encodings_dict.items():
    known_names.extend([name]*len(encs))
    known_encodings.extend(encs)

while True:
    ret, frame = video_capture.read()
    if not ret:
        print('Falha na captura da webcam'); break

    # Reduz resolução p/ acelerar
    small_frame = cv2.resize(frame, (0, 0), fx=0.25, fy=0.25)
    rgb_small_frame = cv2.cvtColor(small_frame, cv2.COLOR_BGR2RGB)

    face_locations = face_recognition.face_locations(rgb_small_frame)
    face_encodings = face_recognition.face_encodings(rgb_small_frame, face_locations)

    for (top, right, bottom, left), face_encoding in zip(face_locations, face_encodings):
        matches = face_recognition.compare_faces(known_encodings, face_encoding, tolerance=0.45)
        name = "Desconhecido"

        face_distances = face_recognition.face_distance(known_encodings, face_encoding)
        if matches:
            best_match_index = face_distances.argmin()
            if matches[best_match_index]:
                name = known_names[best_match_index]

        # Ajusta coordenadas p/ tamanho original
        top, right, bottom, left = [v*4 for v in (top, right, bottom, left)]

        cv2.rectangle(frame, (left, top), (right, bottom), (0, 255, 0), 2)
        cv2.rectangle(frame, (left, bottom - 20), (right, bottom), (0, 255, 0), cv2.FILLED)
        cv2.putText(frame, name, (left + 2, bottom - 5), cv2.FONT_HERSHEY_DUPLEX, 0.5, (0, 0, 0), 1)

    cv2.imshow('Reconhecimento Facial', frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

video_capture.release()
cv2.destroyAllWindows()
